* Sentiment analysis and topic modeling 
* Correlation between topic and sentiment

1. what are you looking for
2. why does it matter
3. how do you know you found what you were looking for

https://developer.twitter.com/en/dashboard

In [1]:
# Import required packages
import json, codecs
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
import time

sns.set(rc={'figure.figsize':(15,12)})

In [2]:
# Load files obtained through twitterscraper 
with codecs.open('harper_tweets.json', 'r', 'utf-8') as harper_tweets:
    harper_tweets = json.load(harper_tweets, encoding='utf-8')

In [3]:
# Extract each element from the tweets and convert them to a pandas DataFrame 
list_tweets = [list(elem.values()) for elem in harper_tweets]
list_columns = list(harper_tweets[0].keys())
df = pd.DataFrame(list_tweets, columns=list_columns)

In [4]:
df.to_csv('harper_tweets.csv')

Since these tweets don't have any location information, we'll need to use the `tweepy` package and have access to the Twitter developers API to extract the location from each tweet id. 

In [4]:
# Import the tweepy package
import tweepy 

# Consumer and access keys that allow access to the Twitter developer API 
consumer_key = ''
consumer_secret = ''

access_token = ''
access_token_secret = ''

# Create the tweepy API 
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [16]:
# %load TweetLocation.py
import tweepy 
import pandas as pd 

class tweet_location:
    """
    Obtains tweet location by passing tweet ids through the parser.
    """

    def __init__(self, id):
        self.id = id

    def get_location(id):
        try:
            tweet_status = api.get_status(id)
            tweet_location = tweet_status.user.location
            tweet_data = [id, tweet_location]
            return(tweet_data)
        except:
            print("Tweet ID not found")
    
    

In [ ]:
if __name__ == '__main__':
    tweet_ids = df['id']
    tweet_loc = []
    
    exec_start_time = time.time()
    
    with Pool(cpu_count()) as p:
    
        chunks = 250
        chunk_size = round(0.5 + len(tweet_ids)/chunks)
    
        for chunk_i in np.arange(0, chunks):
            print("Running chunk {} of {}".format(chunk_i + 1, chunks))
            chunk_start = chunk_i * chunk_size
            chunk_end = chunk_start + chunk_size
            chunk_end = min(len(tweet_ids), chunk_end)
            print("... chunk start: {}".format(chunk_start))
            print("... chunk end: {}".format(chunk_end))

            tmp = p.map(tweet_location.get_location, list(tweet_ids)[chunk_start:chunk_end])
            for i in tmp:
                tweet_location.append(i)

print("Loop took {:.2f} minutes".format((time.time() - exec_start_time)/60.0))

Running chunk 1 of 250
... chunk start: 0
... chunk end: 1161
